In [ ]:
from kogito.models.bart.comet import COMETBART
from kogito.inference import CommonsenseInference

# Load pre-trained knowledge model from HuggingFace
model = COMETBART.from_pretrained()

# Initialize inference module
csi = CommonsenseInference()

# Run inference
text = "Student gets a library card"
context = "library"
kgraph = csi.infer(text, model, context=context)

# Save output knowledge graph to JSON file
kgraph.to_jsonl("kgraph.jsonl")

In [ ]:
from kogito.core.head import KnowledgeHead
from kogito.core.model import KnowledgeModel
from kogito.core.knowledge import Knowledge, KnowledgeGraph
from kogito.core.linker import KnowledgeLinker
from kogito.core.relation import DESIRES
from kogito.linkers.deberta import DeBERTaLinker

# Knowledge representation
knowledge = Knowledge(head=KnowledgeHead("student"),
                      relation=DESIRES,
                      tails=["get good grades"])
input_graph = KnowledgeGraph([knowledge, ...])
input_graph = KnowledgeGraph.from_jsonl("kgraph.jsonl")

# Knowledge models and linkers
model: KnowledgeModel = COMETBART.from_pretrained()
linker: KnowledgeLinker = DeBERTaLinker()

# Train, evaluate, predict and save model
model = model.train(input_graph, ...)
output_graph = model.generate(input_graph, ...)
metrics = model.evaluate(input_graph, ...)
model.save_pretrained(...)

# Run inference
output_graph = csi.infer(text, model=model, linker=linker)

In [ ]:
from typing import List, Tuple
from kogito.core.head import KnowledgeHead
from kogito.core.relation import KnowledgeRelation

from kogito.core.processors.head import KnowledgeHeadExtractor
from kogito.core.processors.relation import KnowledgeRelationMatcher

HeadRelationMatch = Tuple[KnowledgeHead, KnowledgeRelation]

class CustomHeadExtractor(KnowledgeHeadExtractor):
    def extract(self, text, doc) -> List[KnowledgeHead]:
        """your custom head extraction logic"""

class CustomRelationMatcher(KnowledgeRelationMatcher):
    def match(self, heads, relations) -> List[HeadRelationMatch]:
        """your custom relation matching logic"""

class CustomKnowledgeModel(KnowledgeModel):
    def train(self, train_graph, *args, **kwargs) -> KnowledgeModel:
        """your custom training logic"""

    def generate(self, input_graph, *args, **kwargs) -> KnowledgeGraph:
        """your custom inference logic"""

class CustomKnowledgeLinker(KnowledgeLinker):
    def link(self, input_graph, context) -> List[List[float]]:
        """your custom linking logic"""

csi = CommonsenseInference()
custom_extractor = CustomHeadExtractor("custom_extractor")
custom_matcher = CustomRelationMatcher("custom_matcher")
custom_model = CustomKnowledgeModel()
custom_linker = CustomKnowledgeLinker()
csi.add_processor(custom_extractor)
csi.add_processor(custom_matcher)
csi.infer(text, model=custom_model, linker=custom_linker)

In [ ]:
from kogito.inference import CommonsenseInference
from kogito.models.gpt3.zeroshot import GPT3Zeroshot
from kogito.core.relation import KnowledgeRelation, register_relation

# Verbalizer for new relation
def x_wishes_verbalizer(head, **kwargs):
   index = kwargs.get("index")
   index_txt = f"{index}" if index is not None else ""
   return f"Situation {index_txt}: {head}\nWishes: As a result, PersonX wishes"

X_WISHES = KnowledgeRelation("xWishes",
                             verbalizer=x_wishes_verbalizer,
                             prompt="How does this situation affect each character's wishes?")
register_relation(X_WISHES)

# Define sample graph for new relation
sample_graph = KnowledgeGraph.from_csv("sample_graph.tsv", sep="\t")
# Sample graph should contain example knowledge inferences for the new relation
# Example sample graph:
# PersonX is at a party	      xWishes	  to drink beer and dance
# PersonX bleeds a lot	      xWishes	  to see a doctor
# PersonX works as a cashier	xWishes	  to be a store manager
# PersonX gets dirty	         xWishes	  to clean up

# GPT-3 is few-shot prompted with the sample graph
model = GPT3Zeroshot(api_key="", model_name="text-davinci-003")

csi = CommonsenseInference()
kgraph = csi.infer(text, model, sample_graph=sample_graph)

In [ ]:
from kogito.linkers.deberta import DebertaLinker
from kogito.inference import CommonsenseInference

csi = CommonsenseInference()
linker = DebertaLinker()
context = ...
input_graph = ...

# Link input graph to context
relevance_probs = linker.link(input_graph, context)

# Link and filter input graph based on relevancy to the context
filtered_graph = linker.filter(input_graph, context, threshold=0.6)

# Generate inferences and filter based on relevancy to the context
kgraph = csi.infer(text, model, context=context, linker=linker, threshold=0.6)